In [19]:
import sys
!{sys.executable} -m pip install pymilvus==1.1.0

  Using cached pymilvus-1.1.0-py3-none-any.whl (56 kB)
  Attempting uninstall: pymilvus
    Found existing installation: pymilvus 1.0.0
    Uninstalling pymilvus-1.0.0:
      Successfully uninstalled pymilvus-1.0.0
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.16/libexec/bin/python3.9 -m pip install --upgrade pip' command.


In [13]:
import os
import pandas as pd
import numpy as np
import pymysql
from milvus import Milvus, MetricType
from sklearn.preprocessing import StandardScaler

In [122]:
#config settings
STOCK_METRIC = 'Close'
STOCK_WIDTH = 20
STOCK_TIME_STEP = 5

DATASET_PATH = "data/archive/Stocks"
MILVUS_HOST = '127.0.0.1'
MILVUS_PORT = 19530
MILVUS_COLLECTION_NAME = 'stock_historical'
INDEX_FILE_SIZE = 1024
METRIC_TYPE = MetricType.L2

MYSQL_HOST = '127.0.0.1'
MYSQL_PORT = 3306
MYSQL_USER = 'root'
MYSQL_PWD = '123456'
MYSQL_DB = 'mysql'
MYSQL_TABLE = 'milvus_stock_search'

In [3]:
! docker run --name milvus_cpu_1.1.0 -d \
-p 19530:19530 \
-p 19121:19121 \
milvusdb/milvus:1.1.0-cpu-d050721-5e559c

docker: Error response from daemon: Conflict. The container name "/milvus_cpu_1.1.0" is already in use by container "565a57fc80a3467f9be76d93b8ec494121726221c5318fb1fd473d13926390b0". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [119]:
! docker run -p 3306:3306 -e MYSQL_ROOT_PASSWORD=123456 -d mysql:5.7

Unable to find image 'mysql:5.7' locally
5.7: Pulling from library/mysql

81a07f80: Pulling fs layer 
b60610f5: Pulling fs layer 
afb77ab8: Pulling fs layer 
46006037: Pulling fs layer 
e54c8855: Pulling fs layer 
e05cc46d: Pulling fs layer 
814c833f: Pulling fs layer 
5b4af634: Pulling fs layer 
a5b14385: Pulling fs layer 
36297c75: Pulling fs layer 
Digest: sha256:1a2f9cd257e75cc80e9118b303d1648366bc2049101449bf2c8d82b022ea86b72K
Status: Downloaded newer image for mysql:5.7
f8811a1b06b44ac8ef88d680c62c25767cf2920e010827cd46f284ef9b062580
docker: Error response from daemon: driver failed programming external connectivity on endpoint sad_dubinsky (a4b1e8cabca17bf7a20d4d5400c8724a547bc873674f2693bdf0af6b54163dbe): Bind for 0.0.0.0:3306 failed: port is already allocated.


In [145]:
def read_from_file(path):
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
    df['Date'] = df['Date'].apply(lambda x: x.date())
    df.index = df['Date']
    df = df[[STOCK_METRIC]]
    return df

In [146]:
df = read_from_file('data/archive/Stocks/evlmc.us.txt')

In [112]:
def partition_data(df, duration, time_step):
    stock_sets, date_starts, date_ends = [], [], []
    for i in range(0,len(df) - duration,time_step):
        stock_set = df.iloc[i:i + duration]
        scaler = StandardScaler()
        scaled_stock_set = scaler.fit_transform(stock_set)
        stock_sets.append(np.stack(scaled_stock_set, axis=1).tolist()[0])
        date_starts.append(df.index[i])
        date_ends.append(df.index[i])
    return stock_sets, date_starts, date_ends

In [151]:
sets, start, end = partition_data(df, STOCK_WIDTH, STOCK_TIME_STEP)
type(sets[0])
#sets

list

In [129]:
def load_data_to_mysql(data):
    sql = "insert into " + MYSQL_TABLE + " (milvus_id,file_path,start_date,end_date) values (%s,%s,%s,%s);"
    cursor.executemany(sql, data)
    conn.commit()

In [115]:
client = Milvus(host=MILVUS_HOST, port=MILVUS_PORT)
try:
    if not client.has_collection(MILVUS_COLLECTION_NAME)[1]:
        collection_param = {
            'collection_name': MILVUS_COLLECTION_NAME,
            'dimension': STOCK_WIDTH,
            'index_file_size': INDEX_FILE_SIZE,
            'metric_type': METRIC_TYPE
        }
        status = client.create_collection(collection_param)
        if status.code != 0:
            raise Exception(status.message)
except Exception as e:
    print("Failed to load data to Milvus: {}".format(e))

In [161]:
conn = pymysql.connect(host=MYSQL_HOST, user=MYSQL_USER, port=MYSQL_PORT, 
                       password=MYSQL_PWD, database=MYSQL_DB, local_infile=True)
cursor = conn.cursor()
sql = "create table if not exists " + MYSQL_TABLE + "(milvus_id TEXT, file_path TEXT, start_date DATE, end_date DATE);"
try:
    cursor.execute(sql)
except Exception as e:
    print("MYSQL ERROR: {} with sql: {}".format(e, sql))

In [168]:
directory = os.fsencode(DATASET_PATH)
for file in os.listdir(directory):
#for file in ['data/archive/Stocks/evlmc.us.txt']:
    filename = os.fsdecode(file)
    if filename.endswith(".txt"): 
        try:
            #full_path = os.path.join(directory, file)
            full_path = DATASET_PATH + '/' + filename
            #full_path = 'data/archive/Stocks/evlmc.us.txt'
            #print(full_path)
            df = read_from_file(full_path)
            sets, start, end = partition_data(df, STOCK_WIDTH, STOCK_TIME_STEP)
            #print(type(start[0]))
            status, ids = client.insert(collection_name=MILVUS_COLLECTION_NAME, records=sets)
            if not status.OK():
                print("Insert failed for file {}: {}".format(full_path, status))
                continue
            ids = list(map(str, ids))
            
            load_data_to_mysql(zip(ids, [full_path] * len(ids), start, end))
            
            
            break
        except Exception as e:
            print("Failed to load data: {}".format(e))
            continue
    else:
        continue

Status(code=0, message='Add vectors successfully!')


In [167]:
cursor.execute("SELECT * FROM milvus_stock_search;")
result = cursor.fetchall()
print(result)

(('1624924415187576000', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 4, 1), datetime.date(2016, 4, 1)), ('1624924415187576001', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 4, 7), datetime.date(2016, 4, 7)), ('1624924415187576002', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 4, 13), datetime.date(2016, 4, 13)), ('1624924415187576003', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 4, 19), datetime.date(2016, 4, 19)), ('1624924415187576004', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 4, 25), datetime.date(2016, 4, 25)), ('1624924415187576005', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 4, 30), datetime.date(2016, 4, 30)), ('1624924415187576006', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 5, 6), datetime.date(2016, 5, 6)), ('1624924415187576007', 'data/archive/Stocks/evlmc.us.txt', datetime.date(2016, 5, 12), datetime.date(2016, 5, 12)), ('1624924415187576008', 'data/archive/Stocks/evlmc.us.txt', datetime.

In [ ]:
cursor.execute("SELECT * FROM geeksdemo")
  
# fetch all the matching rows 